<center>
<h1 style="font-size: 50px; font-weight: bold; color:sandybrown">OC SWITRS R Data Processing</h1>

<div style="font-size: 40px; font-weight: bold; color: sandybrown">ArcGIS and Spatial Geoprocessing</div>
<div style="font-size: 30px; font-weight: bold; color: sandybrown">v.1, January 2025</div>
</center>

----

In [ ]:
# %reset

## <font color="orangered">**1. Referencing Libraries and Initialization**</font>

### <font color="blue">**1.1. Preliminaries**</font>

Instantiating python libraries for the project

In [1]:
# Import Python libraries
import os
import json
import pytz
import math
import arcpy
import arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from tqdm.notebook import trange, tqdm, tqdm_notebook
import pandas as pd
import numpy as np
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor


### <font color="blue">**1.2. Project and Workspace Variables**</font>

Define and maitnain project, workspace, ArcGIS, and data-related variables

#### <font color="darkgreen">Project and Geodatabase Paths</font>

Project and ArcGIS pro project paths

In [2]:
# Environment variables for OneDrive path
onedrive_path = os.getenv("OneDriveCommercial")

# OC Switrs project path
project_path = os.path.join(onedrive_path, "Documents", "OCSWITRS")

# OC Switrs ArcGIS Pro project path
agp_path = os.path.join(project_path, "AGPSWITRS")

print(f"Project Directories:\n- Project Path:{project_path}\n- ArcGIS Pro Project Path: {agp_path}")


Project Directories:
- Project Path:C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS
- ArcGIS Pro Project Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS


#### <font color="darkgreen">ArcGIS Pro Paths</font>

ArcGIS pro related paths

In [3]:
# ArcGIS Pro project name and path
project_aprx = "AGPSWITRS.aprx"
project_aprx_path = os.path.join(agp_path, project_aprx)

# ArcGIS Pro project geodatabase and path
gdb = "AGPSWITRS.gdb"
gdb_path = os.path.join(agp_path, gdb)

# Current ArcGIS pro project
aprx = arcpy.mp.ArcGISProject('CURRENT')

# Close all map views
aprx.closeViews()

# Current ArcGIS pro project maps
map_main = aprx.listMaps("MainMap")[0]
map_hotspots = aprx.listMaps("HotSpotsMap")[0]

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdb_path
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True


#### <font color="darkgreen">Folder Paths</font>

Project folder paths

In [4]:
# Raw data folder path
raw_data_path = os.path.join(project_path, "Data", "Raw")

# Layers folder path
layers_path = os.path.join(project_path, "Layers")

# Notebooks folder path
notebooks_path = os.path.join(project_path, "Notebooks")

# Geodatabase feature directory paths
gdb_raw_data_path = os.path.join(workspace, "RawData") # raw data
gdb_supporting_data_path = os.path.join(workspace, "SupportingData") # supporting data
gdb_analysis_data_path = os.path.join(workspace, "AnalysisData") # analysis data
gdb_hotspot_data_path = os.path.join(workspace, "HotSpotData") # hotspot data


# Codebook Json folder path
codebook_path = os.path.join(project_path, "Data", "codebook", "cb.json")


#### <font color="darkgreen">Data Folder Paths</font>

Data folder paths and contents

The most current raw data files cover the periods from 01/01/2013 to 09/30/2024. The data files are already processed in the R scripts and imported into the project's geodatabase.

In [5]:
# add the date 01/01/2013 to a new python datetime object named 'rawDateStart'
raw_date_start = datetime(2013, 1, 1)

# add the date 06/30/2024 to a new python datetime object named 'rawDateEnd'
raw_date_end = datetime(2024, 9, 30)

# Define time and date variables
time_zone = pytz.timezone("US/Pacific")
today = datetime.now(time_zone)
last_update_date = today.strftime("%B %d, %Y")
last_update_datetime = today.strftime("%A %B %d, %Y, %I:%M %p (%Z)")


#### <font color="darkgreen">Geodatabase Feature Classes Paths</font>

Paths to geodatabase feature classes

In [6]:
# Paths to raw data feature classes
crashes_path = os.path.join(gdb_raw_data_path, "crashes")
parties_path = os.path.join(gdb_raw_data_path, "parties")
victims_path = os.path.join(gdb_raw_data_path, "victims")
collisions_path = os.path.join(gdb_raw_data_path, "collisions")

# Paths to supporting data feature classes
boundaries_path = os.path.join(gdb_supporting_data_path, "boundaries")
cities_path = os.path.join(gdb_supporting_data_path, "cities")
roads_path = os.path.join(gdb_supporting_data_path, "roads")
usc2020blk_path = os.path.join(gdb_supporting_data_path, "usc2020blk")


Display all information about paths, folders, and data files

In [7]:
# Display all information
print("Key Project Information")
print(f"\n\t- Name {project_aprx}\n\t- Path: {project_aprx_path}\n\t- Project Path: {project_path}\n\t- Workspace: {workspace}\n\t- Geodatabase: {gdb}\n\t- Geodatabase Path: {gdb_path}")
print("\nProject Directories:")
print(
    f"\n\t- Raw Data: {raw_data_path}\n\t- Layers: {layers_path}\n\t- Notebooks: {notebooks_path}")
print("\nRaw Feature Classes:")
print(f"\n\t- Crashes: {crashes_path}\n\t- Parties: {parties_path}\n\t- Victims: {victims_path}\n\t- Collisions: {collisions_path}")
print("\nSupporting Feature Classes:")
print(
    f"\n\t- Boundaries: {boundaries_path}\n\t- Cities: {cities_path}\n\t- Roads: {roads_path}\n\t- USC 2020 Blocks: {usc2020blk_path}")
print("\nOther Supporting Data")
print(f"\n\t- Codebook: {codebook_path}")


Key Project Information

	- Name AGPSWITRS.aprx
	- Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.aprx
	- Project Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS
	- Workspace: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb
	- Geodatabase: AGPSWITRS.gdb
	- Geodatabase Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb

Project Directories:

	- Raw Data: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Data\Raw
	- Layers: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Layers
	- Notebooks: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Notebooks

Raw Feature Classes:

	- Crashes: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawData\crashes
	- Parties: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawDa

#### <font color="darkgreen">Current Map Operations</font>

Remove all layers from main map

In [8]:
# Add and rename layers in contents view, using their aliases
for lyr in map_main.listLayers():
    if "OCSWITRS" in lyr.name:
        map_main.removeLayer(lyr)

Open the main map view, and set it as the active map

In [9]:
# Open the main map window
map_main.openView()

# set the main map as active map
map = aprx.activeMap

## <font color="orangered">**2. Feature Classes and Data Definitions**</font>

#### <font color="darkgreen">Codebook</font>

Codebook JSON file loading

In [10]:
# Load the JSON file from directory and store it in a variable
with open(codebook_path) as json_file:
    codebook = json.load(json_file)


#### <font color="darkgreen">Paths</font>

Set the raw data file path as the default and read feature classes

In [11]:
arcpy.env.workspace = gdb_raw_data_path
workspace = arcpy.env.workspace
arcpy.ListFeatureClasses()


['crashes', 'parties', 'victims', 'collisions']

Set the path as the root of the geodatabase

In [12]:
arcpy.env.workspace = gdb_path
workspace = arcpy.env.workspace


#### <font color="darkgreen">Feature Classes and Fields</font>

In [13]:
# Get all the feature classes by name
boundaries_fc = os.path.join(gdb_supporting_data_path, arcpy.ListFeatureClasses("boundaries", feature_dataset="SupportingData")[0])
cities_fc = os.path.join(gdb_supporting_data_path, arcpy.ListFeatureClasses("cities", feature_dataset="SupportingData")[0])
usc2020blk_fc = os.path.join(gdb_supporting_data_path, arcpy.ListFeatureClasses("usc2020blk", feature_dataset="SupportingData")[0])
roads_fc = os.path.join(gdb_supporting_data_path, arcpy.ListFeatureClasses("roads", feature_dataset="SupportingData")[0])
victims_fc = os.path.join(gdb_raw_data_path, arcpy.ListFeatureClasses("victims", feature_dataset="RawData")[0])
parties_fc = os.path.join(gdb_raw_data_path, arcpy.ListFeatureClasses("parties", feature_dataset="RawData")[0])
crashes_fc = os.path.join(gdb_raw_data_path, arcpy.ListFeatureClasses("crashes", feature_dataset="RawData")[0])
collisions_fc = os.path.join(gdb_raw_data_path, arcpy.ListFeatureClasses("collisions", feature_dataset="RawData")[0])


Obrain the list fields of the geodatabase data

In [14]:
# Field lists for each of the geodatabase raw data feature classes
fields_collisions = arcpy.ListFields(collisions_path)
fields_crashes = arcpy.ListFields(crashes_path)
fields_parties = arcpy.ListFields(parties_path)
fields_victims = arcpy.ListFields(victims_path)

# Field lists for each of the geodatabase supporting data feature classes
fields_roads = arcpy.ListFields(roads_path)
fields_usc2020blk = arcpy.ListFields(usc2020blk_path)
fields_cities = arcpy.ListFields(cities_path)
fields_boundaries = arcpy.ListFields(boundaries_path)


Count rows in each of the geodatabase feature classes

In [15]:
# Get the count for each of the feature classes
count_collisions = int(arcpy.management.GetCount(collisions_fc)[0])
count_crashes = int(arcpy.management.GetCount(crashes_fc)[0])
count_victims = int(arcpy.management.GetCount(victims_fc)[0])
count_parties = int(arcpy.management.GetCount(parties_fc)[0])
count_roads = int(arcpy.management.GetCount(roads_fc)[0])
count_usc2020blk = int(arcpy.management.GetCount(usc2020blk_fc)[0])
count_cities = int(arcpy.management.GetCount(cities_fc)[0])
count_boundaries = int(arcpy.management.GetCount(boundaries_fc)[0])

print(f"Counts:\n- Collisions: {count_collisions:,}\n- Crashes: {count_crashes:,}\n- Parties: {count_parties:,}\n- Victims: {count_victims:,}\n- Roads: {count_roads:,}\n- US Census 2020 Blocks: {count_usc2020blk}\n- Cities: {count_cities:,}\n- Boundaries: {count_boundaries:,}")


Counts:
- Collisions: 391,782
- Crashes: 152,987
- Parties: 330,435
- Victims: 264,954
- Roads: 59,144
- US Census 2020 Blocks: 26864
- Cities: 46
- Boundaries: 1


## <font color="orangered">**3. Codebook for Field Aliases**</font>

### <font color="blue">**3.1. Assigning field aliases**</font>

Creating and assigning field aliases for the geodatabase feature classes using the JSON codebook dictionary

#### <font color="darkgreen">Crashes Feature Class</font>

Field aliases for the crashes feature class

In [16]:
# Field aliases for the crashes geodatabase feature class
for field in fields_crashes:
    if field.name in list(codebook.keys()):
        print(
            f"\tMatch {codebook[field.name]['var_order']}: {field.name} ({codebook[field.name]['label']})")
        arcpy.management.AlterField(
            in_table = crashes_fc,
            field = field.name,
            new_field_alias = codebook[field.name]['label']
        )
print(arcpy.GetMessages())


	Match 1: case_id (Case ID)
	Match 2: cid (Crash ID)
	Match 5: crash_tag (Crash Tag)
	Match 10: crashes_case_tag (Crashes Tag)
	Match 17: crashes_cid_count (Total Collisions in Crashes Dataset)
	Match 23: city (City)
	Match 25: date_datetime (Crash Date and Time)
	Match 26: date_year (Crash Year)
	Match 27: date_quarter (Crash Quarter)
	Match 28: date_month (Crash Month)
	Match 29: date_week (Crash Week)
	Match 30: date_day (Crash Day)
	Match 31: date_process (Process Date)
	Match 32: dt_year (Crash Year)
	Match 33: dt_quarter (Crash Quarter)
	Match 34: dt_month (Crash Month)
	Match 35: dt_year_week (Crash Year Week)
	Match 36: dt_week_day (Crash Weekday)
	Match 37: dt_month_day (Crash Month Day)
	Match 38: dt_year_day (Crash Year Day)
	Match 39: dt_hour (Crash Hour)
	Match 40: dt_minute (Crash Minute)
	Match 41: dt_dst (Crash Daylight Saving Time)
	Match 42: dt_zone (Crash Time Zone)
	Match 43: coll_date (Collision Date (reported))
	Match 44: coll_time (Collision Time (reported))
	Mat

#### <font color="darkgreen">Parties Feature Class</font>

Field aliases for the parties feature class

In [17]:
# Field aliases for the parties geodatabase feature class
for field in fields_parties:
    if field.name in list(codebook.keys()):
        print(
            f"\tMatch {codebook[field.name]['var_order']}: {field.name} ({codebook[field.name]['label']})")
        arcpy.management.AlterField(
            in_table = parties_fc,
            field = field.name,
            new_field_alias = codebook[field.name]['label']
        )
print(arcpy.GetMessages())


	Match 1: case_id (Case ID)
	Match 2: cid (Crash ID)
	Match 3: pid (Party ID)
	Match 6: party_tag (Party Tag)
	Match 8: party_number (Party Number)
	Match 11: parties_case_tag (Parties Tag)
	Match 18: parties_cid_count (Total Collisions in Parties Dataset)
	Match 20: parties_pid_count (Total Parties in Parties Dataset)
	Match 25: date_datetime (Crash Date and Time)
	Match 26: date_year (Crash Year)
	Match 27: date_quarter (Crash Quarter)
	Match 28: date_month (Crash Month)
	Match 29: date_week (Crash Week)
	Match 30: date_day (Crash Day)
	Match 31: date_process (Process Date)
	Match 32: dt_year (Crash Year)
	Match 33: dt_quarter (Crash Quarter)
	Match 34: dt_month (Crash Month)
	Match 35: dt_year_week (Crash Year Week)
	Match 36: dt_week_day (Crash Weekday)
	Match 37: dt_month_day (Crash Month Day)
	Match 38: dt_year_day (Crash Year Day)
	Match 39: dt_hour (Crash Hour)
	Match 40: dt_minute (Crash Minute)
	Match 41: dt_dst (Crash Daylight Saving Time)
	Match 42: dt_zone (Crash Time Zone

#### <font color="darkgreen">Victims Feature Class</font>

Field aliases for the victims feature class

In [18]:
# Field aliases for the victims geodatabase feature class
for field in fields_victims:
    if field.name in list(codebook.keys()):
        print(
            f"\tMatch {codebook[field.name]['var_order']}: {field.name} ({codebook[field.name]['label']})")
        arcpy.management.AlterField(
            in_table = victims_fc,
            field = field.name,
            new_field_alias = codebook[field.name]['label']
        )
print(arcpy.GetMessages())


	Match 1: case_id (Case ID)
	Match 2: cid (Crash ID)
	Match 3: pid (Party ID)
	Match 4: vid (Victim ID)
	Match 7: victim_tag (Victim Tag)
	Match 8: party_number (Party Number)
	Match 9: victim_number (Victim Number)
	Match 12: victims_case_tag (Victims Tag)
	Match 19: victims_cid_count (Total Collisions in Victims Dataset)
	Match 21: victims_pid_count (Total Parties in Victims Dataset)
	Match 22: victims_vid_count (Total Victims in Victims Dataset)
	Match 25: date_datetime (Crash Date and Time)
	Match 26: date_year (Crash Year)
	Match 27: date_quarter (Crash Quarter)
	Match 28: date_month (Crash Month)
	Match 29: date_week (Crash Week)
	Match 30: date_day (Crash Day)
	Match 31: date_process (Process Date)
	Match 32: dt_year (Crash Year)
	Match 33: dt_quarter (Crash Quarter)
	Match 34: dt_month (Crash Month)
	Match 35: dt_year_week (Crash Year Week)
	Match 36: dt_week_day (Crash Weekday)
	Match 37: dt_month_day (Crash Month Day)
	Match 38: dt_year_day (Crash Year Day)
	Match 39: dt_hour

#### <font color="darkgreen">Collisions Feature Class</font>

Field aliases for the collisions feature class

In [19]:
# Field aliases for the collisions geodatabase feature class
for field in fields_collisions:
    if field.name in list(codebook.keys()):
        print(
            f"\tMatch {codebook[field.name]['var_order']}: {field.name} ({codebook[field.name]['label']})")
        arcpy.management.AlterField(
            in_table = collisions_fc,
            field = field.name,
            new_field_alias = codebook[field.name]['label']
        )
print(arcpy.GetMessages())


	Match 1: case_id (Case ID)
	Match 2: cid (Crash ID)
	Match 3: pid (Party ID)
	Match 4: vid (Victim ID)
	Match 5: crash_tag (Crash Tag)
	Match 6: party_tag (Party Tag)
	Match 7: victim_tag (Victim Tag)
	Match 8: party_number (Party Number)
	Match 9: victim_number (Victim Number)
	Match 10: crashes_case_tag (Crashes Tag)
	Match 11: parties_case_tag (Parties Tag)
	Match 12: victims_case_tag (Victims Tag)
	Match 16: combined_ind (Combined Indicator)
	Match 17: crashes_cid_count (Total Collisions in Crashes Dataset)
	Match 18: parties_cid_count (Total Collisions in Parties Dataset)
	Match 19: victims_cid_count (Total Collisions in Victims Dataset)
	Match 20: parties_pid_count (Total Parties in Parties Dataset)
	Match 21: victims_pid_count (Total Parties in Victims Dataset)
	Match 22: victims_vid_count (Total Victims in Victims Dataset)
	Match 23: city (City)
	Match 24: place_type (Place Type)
	Match 25: date_datetime (Crash Date and Time)
	Match 26: date_year (Crash Year)
	Match 27: date_q

#### <font color="darkgreen">Cities Feature Class</font>

Field aliases for the cities feature class

In [20]:
# Field aliases for the cities geodatabase feature class
for field in fields_cities:
    if field.name in list(codebook.keys()):
        print(
            f"\tMatch {codebook[field.name]['var_order']}: {field.name} ({codebook[field.name]['label']})")
        arcpy.management.AlterField(
            in_table = cities_fc,
            field = field.name,
            new_field_alias = codebook[field.name]['label']
        )
print(arcpy.GetMessages())


	Match 23: city (City)
	Match 24: place_type (Place Type)
	Match 164: city_area_sqmi (City Area (in sq miles))
	Match 167: city_pop_total (City Total Population)
	Match 168: city_hou_total (City Total Housing Units)
	Match 165: city_pop_dens (City Population Density (sq miles))
	Match 166: city_hou_dens (City Housing Density (sq miles))
	Match 169: city_pop_asian (City Asian Population)
	Match 170: city_pop_black (City Black Population)
	Match 171: city_pop_hispanic (City Hispanic Population)
	Match 172: city_pop_white (City White Population)
	Match 173: city_vehicles (Number of Commuting Vehicles)
	Match 174: city_travel_time (Aggregate Travel Time to Work (min))
	Match 175: city_mean_travel_time (Mean Travel Time to work Per Vehicle (min))
Start Time: Saturday, January 25, 2025 9:11:46 AM
Succeeded at Saturday, January 25, 2025 9:11:46 AM (Elapsed Time: 0.45 seconds)


#### <font color="darkgreen">Roads Feature Class</font>

Field aliases for the roads feature class

In [21]:
# Field aliases for the roads geodatabase feature class
for field in fields_roads:
    if field.name in list(codebook.keys()):
        print(f"\tMatch {codebook[field.name]['var_order']}: {field.name} ({codebook[field.name]['label']})")
        arcpy.management.AlterField(
            in_table = roads_fc,
            field = field.name,
            new_field_alias = codebook[field.name]['label']
        )
print(arcpy.GetMessages())


	Match 178: place_name (Road City)
	Match 176: road_name (Road Name)
	Match 177: road_id (Road ID)
	Match 179: road_cat (Road Category)
	Match 180: road_length (Road Length (in miles))
	Match 181: roads_primary (Number of Primary Road Segments)
	Match 182: roads_secondary (Number of Secondary Road Segments)
	Match 183: roads_local (Number of Local Road Segments)
	Match 184: road_length_mean (Mean Road Length (in miles))
	Match 185: road_length_sum (Total Road Length (in miles))
Start Time: Saturday, January 25, 2025 9:11:58 AM
Succeeded at Saturday, January 25, 2025 9:11:58 AM (Elapsed Time: 0.45 seconds)


## <font color="orangered">**4. Geodatabase Metadata Operations**</font>

### <font color="blue">**4.1. Feature Class Aliases**</font>

Define name aliases for the feature classes (to be used both for defining feature class aliases, and for calling layer names in map contents)

In [22]:
# Define aliases for each of the feature classes
alias_collisions = "OCSWITRS Collisions"
alias_crashes = "OCSWITRS Crashes"
alias_parties = "OCSWITRS Parties"
alias_victims = "OCSWITRS Victims"
alias_roads = "OCSWITRS Roads"
alias_usc2020blk = "OCSWITRS Census 2020 Blocks"
alias_cities = "OCSWITRS Cities"
alias_boundaries = "OCSWITRS Boundaries"


Assign aliases to feature classes

In [23]:
# Assign alias operations to each of the feature classes
arcpy.AlterAliasName(collisions_path, alias_collisions)
arcpy.AlterAliasName(crashes_path, alias_crashes)
arcpy.AlterAliasName(parties_path, alias_parties)
arcpy.AlterAliasName(victims_path, alias_victims)
arcpy.AlterAliasName(roads_path, alias_roads)
arcpy.AlterAliasName(usc2020blk_path, alias_usc2020blk)
arcpy.AlterAliasName(cities_path, alias_cities)
arcpy.AlterAliasName(boundaries_path, alias_boundaries)


Save the ArcGIS pro project

In [24]:
# Save the project
aprx.save()


Define date strings for metadata

In [25]:
# String defining the years of the raw data
md_years = f"{raw_date_start.year}-{raw_date_end.year}"

# String defining the start and end dates of the raw data
md_dates = f"{raw_date_start.strftime('%B %d, %Y')} to {raw_date_end.strftime('%B %d, %Y')}"
print(f"{md_years} ({md_dates})")


2013-2024 (January 01, 2013 to September 30, 2024)


### <font color="blue">**4.2. Collisions Metadata**</font>

Create a collisions metadata object

In [26]:
# Define key metadata attributes for the Collisions feature class
mdo_collisions = md.Metadata()
mdo_collisions.title = "OCSWITRS Combined Collisions Points"
mdo_collisions.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdo_collisions.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({md_years})"
mdo_collisions.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {md_years} ({md_dates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{last_update_date}</b></span></p></div></div></div>"""
mdo_collisions.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_collisions.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_collisions.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"

Assign the collisions metadata object to the collisions feature class

In [27]:
# Apply the metadata object to the collisions feature class
md_collisions = md.Metadata(collisions_fc)
if not md_collisions.isReadOnly:
    md_collisions.copy(mdo_collisions)
    md_collisions.save()

### <font color="blue">**4.3. Crashes Metadata**</font>

Create a new *Crashes* metadata object

In [28]:
# Define key metadata attributes for the Crashes feature class
mdo_crashes = md.Metadata()
mdo_crashes.title = "OCSWITRS Crashes Points"
mdo_crashes.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdo_crashes.summary = f"Statewide Integrated Traffic Records System (SWITRS) Crash Data for Orange County, California ({md_years})"
mdo_crashes.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on crashes</span><span> in Orange County, California for {md_years} ({md_dates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{
    last_update_date}</b></span></p></div></div></div>"""
mdo_crashes.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_crashes.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_crashes.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"


Assign the Crashes metadata object to the crashes feature class

In [29]:
# Apply the metadata object to the crashes feature class
md_crashes = md.Metadata(crashes_fc)
if not md_crashes.isReadOnly:
    md_crashes.copy(mdo_crashes)
    md_crashes.save()


### <font color="blue">**4.4. Parties Metadata**</font>

Create a parties metadata object

In [30]:
# Define key metadata attributes for the Parties feature class
mdo_parties = md.Metadata()
mdo_parties.title = "OCSWITRS Parties Points"
mdo_parties.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Parties, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdo_parties.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Parties Data for Orange County, California ({md_years})"
mdo_parties.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on parties involved in crash incidents</span><span> in Orange County, California for {md_years} ({md_dates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{
    last_update_date}</b></span></p></div></div></div>"""
mdo_parties.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_parties.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_parties.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/1e07bb1002f9457fa6fd3540fdb08e29/data"


Assign the parties metadata object to the parties feature class

In [31]:
# Apply the metadata object to the parties feature class
md_parties = md.Metadata(parties_fc)
if not md_parties.isReadOnly:
    md_parties.copy(mdo_parties)
    md_parties.save()


### <font color="blue">**4.5. Victims Metadata**</font>

Create a victims metadata object

In [32]:
# Define key metadata attributes for the Victims feature class
mdo_victims = md.Metadata()
mdo_victims.title = "OCSWITRS Victims Points"
mdo_victims.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Victims, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdo_victims.summary = f"Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Victims Data for Orange County, California ({md_years})"
mdo_victims.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on victims/persons involved in crash incidents</span><span> in Orange County, California for {md_years} ({md_dates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{
    last_update_date}</b></span></p></div></div></div>"""
mdo_victims.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_victims.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_victims.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/78682395df4744009c58625f1db0c25b/data"


Assign the victims metadata object to the victims feature class

In [33]:
# Apply the metadata object to the victims feature class
md_victims = md.Metadata(victims_fc)
if not md_victims.isReadOnly:
    md_victims.copy(mdo_victims)
    md_victims.save()


### <font color="blue">**4.6. Roads Metadata**</font>

Create a roads metadata object

In [34]:
# Define key metadata attributes for the Roads feature class
mdo_roads = md.Metadata()
mdo_roads.title = "OCSWITRS Roads Network"
mdo_roads.tags = "Orange County, California, Roads, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdo_roads.summary = "All roads for Orange County, California (Primary roads and highways, secondary roads, and local roads)"
mdo_roads.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County Roads Network is a comprehensive representation of all roads in the area, including primary roads and highways, secondary roads, and local roads. The data are sourced from the Orange County Department of Public Works and are updated regularly to reflect the most current road network configuration.</span></p></div></div></div>"""
mdo_roads.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_roads.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_roads.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/76f6fbe9acbb482c9684307854d6352b/data"


Assign the roads metadata object to the roads feature class

In [35]:
# Apply the metadata object to the roads feature class
md_roads = md.Metadata(roads_fc)
if not md_roads.isReadOnly:
    md_roads.copy(mdo_roads)
    md_roads.save()


### <font color="blue">**4.7. US Census 2020 Blocks Metadata**</font>

Create a US Census 2020 Blocks metadata object

In [36]:
# Define key metadata attributes for the US Census 2020 Blocks feature class
mdo_usc2020blk = md.Metadata()
mdo_usc2020blk.title = "OCSWITRS US Census 2020 Blocks"
mdo_usc2020blk.tags = "Orange County, California, US Census 2020, Blocks, Census, Demographics, Population"
mdo_usc2020blk.summary = "US Census 2020 Blocks for Orange County, California"
mdo_usc2020blk.description = """<div style="text-align:Left;"><div><div><p><span>The US Census 2020 Blocks feature class provides a comprehensive representation of the 2020 Census Blocks for Orange County, California. The data are sourced from the US Census Bureau and are updated regularly to reflect the most current demographic and population data.</span></p></div></div></div>"""
mdo_usc2020blk.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_usc2020blk.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_usc2020blk.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/e2c4cd39783a4d1bb0925ead15a23cdc/data"


Assign the US Census 2020 Blocks metadata object to the US Census 2020 Blocks feature class

In [37]:
# Apply the metadata object to the US Census 2020 Blocks feature class
md_usc2020blk = md.Metadata(usc2020blk_fc)
if not md_usc2020blk.isReadOnly:
    md_usc2020blk.copy(mdo_usc2020blk)
    md_usc2020blk.save()
    

### <font color="blue">**4.8. Cities Metadata**</font>

Create a cities metadata object

In [38]:
# Define key metadata attributes for the Cities feature class
mdo_cities = md.Metadata()
mdo_cities.title = "OCSWITRS Cities Boundaries"
mdo_cities.tags = "Orange County, California, Cities, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdo_cities.summary = "Orange County City and Unincorporated Areas Land Boundaries, enriched with geodemographic characteristics"
mdo_cities.description = """<div style="text-align:Left;"><div><div><p><span>The Orange County City and Unincorporated Areas Land Boundaries are enriched with a comprehensive set of geodemographic characteristics from OC ACS 2021 data. These characteristics span across demographic, housing, economic, and social aspects, providing a holistic view of the area. </span></p><p><span>The geodemographic data originate from the US Census American Community Survey (ACS) 2021, a 5-year estimate of the key Characteristics of Cities' geographic level in Orange County, California. The data contains:</span></p><ul><li><p><span>Total population and housing counts for each area;</span></p></li><li><p><span>Population and housing density measurements (per square mile);</span></p></li><li><p><span>Race counts for Asian, Black or African American, Hispanic and White groups;</span></p></li><li><p><span>Aggregate values for the number of vehicles commuting and travel time to work;</span></p></li></ul></div></div></div>"""
mdo_cities.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_cities.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_cities.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/ffe4a73307a245eda7dc7eaffe1db6d2/data"


Assign the cities metadata object to the cities feature class

In [39]:
# Apply the metadata object to the cities feature class
md_cities = md.Metadata(cities_fc)
if not md_cities.isReadOnly:
    md_cities.copy(mdo_cities)
    md_cities.save()


### <font color="blue">**4.9. Boundaries Metadata**</font>

Create a boundaries metadata object

In [40]:
# Define key metadata attributes for the Boundaries feature class
mdo_boundaries = md.Metadata()
mdo_boundaries.title = "OC Land Boundaries"
mdo_boundaries.tags = "Orange County, California, Boundaries, Traffic, Road Safety, Transportation, Collisions, Crashes, SWITRS, OCSWITRS"
mdo_boundaries.summary = "Land boundaries for Orange County, cities, and unincorporated areas"
mdo_boundaries.description = """<div style="text-align:Left;"><div><div><p><span>Land boundaries for Orange County, cities, and unincorporated areas (based on the five supervisorial districts). Contains additional geodemographic data on population and housing from the US Census 2021 American Community Survey (ACS).</span></p></div></div></div>"""
mdo_boundaries.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdo_boundaries.accessConstraints = """<div style="text-align:Left;"><div><div><p><span>The feed data and associated resources (maps, apps, endpoints) can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdo_boundaries.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/4041c4b1f4234218a4ce654e5d22f176/data"


Assign the boundaries metadata object to the boundaries feature class

In [41]:
# Apply the metadata object to the boundaries feature class
md_boundaries = md.Metadata(boundaries_fc)
if not md_boundaries.isReadOnly:
    md_boundaries.copy(mdo_boundaries)
    md_boundaries.save()


Save the ArcGIS pro project

In [42]:
# Save the project
aprx.save()


## <font color="orangered">**5. Map Layer Operations**</font>

In this section we will be creating map layers for the feature classes.

### <font color="blue">**5.1. Visual Layer Operations**</font>

Add and rename layers in contents view, using their aliases

Add the feature classes to the map

In [43]:
# Add the layers to the map (in order, the first layer goes to the bottom of the contents)
boundaries_lyr_name = map_main.addDataFromPath(boundaries_fc)
cities_lyr_name = map_main.addDataFromPath(cities_fc)
usc2020blk_lyr_name = map_main.addDataFromPath(usc2020blk_fc)
roads_lyr_name = map_main.addDataFromPath(roads_fc)
victims_lyr_name = map_main.addDataFromPath(victims_fc)
parties_lyr_name = map_main.addDataFromPath(parties_fc)
crashes_lyr_name = map_main.addDataFromPath(crashes_fc)
collisions_lyr_name = map_main.addDataFromPath(collisions_fc)

print(f"Added layers to the map:\n- {collisions_lyr_name}\n- {crashes_lyr_name}\n- {parties_lyr_name}\n- {victims_lyr_name}\n- {roads_lyr_name}\n- {usc2020blk_lyr_name}\n- {cities_lyr_name}\n- {boundaries_lyr_name}")


Added layers to the map:
- OCSWITRS Collisions
- OCSWITRS Crashes
- OCSWITRS Parties
- OCSWITRS Victims
- OCSWITRS Roads
- OCSWITRS Census 2020 Blocks
- OCSWITRS Cities
- OCSWITRS Boundaries


In [44]:
# Save the project
aprx.save()


Define individual layers for each of the SWITRS feature classes

In [45]:
# Store the current list of layers to a new variable
map_lyrs = map_main.listLayers()

# Store the names of the layers in a list
map_lyrs_list = [mlyr.name for mlyr in map_lyrs]

print("Active Map Layers:")
# Loop through the list of layers in the map, and print their names
for mlyr in map_lyrs:
    print(f" - {mlyr.name}")


Active Map Layers:
 - OCSWITRS Collisions
 - OCSWITRS Crashes
 - OCSWITRS Parties
 - OCSWITRS Victims
 - OCSWITRS Roads
 - OCSWITRS Census 2020 Blocks
 - OCSWITRS Cities
 - OCSWITRS Boundaries
 - World Topographic Map


In [46]:
# Identify each of the Crashes, Parties, Victim and Collisions layers
collisions_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_collisions), False)
crashes_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_crashes), False)
parties_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_parties), False)
victims_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_victims), False)
roads_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_roads), False)
usc2020blk_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_usc2020blk), False)
cities_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_cities), False)
boundaries_lyr = next((mlyr for mlyr in map_lyrs if mlyr.name == alias_boundaries), False)

# Define a list of the four layers, and a list of supporting data layers
data_lyrs_list = [collisions_lyr, crashes_lyr, parties_lyr, victims_lyr]
supp_lyrs_list = [roads_lyr, usc2020blk_lyr, cities_lyr, boundaries_lyr]


Turn off the initial visibility of all layers in the map

In [47]:
# Iterate through the data layers list
for lyr in data_lyrs_list:
    # If the layer is not False
    if lyr:
        # Set the visibility of the layer to True
        lyr.visible = False

# Iterate through the supporting data layers list
for lyr in supp_lyrs_list:
    # If the layer is not False
    if lyr:
        # Set the visibility of the layer to True
        lyr.visible = False


Save the ArcGIS pro project

In [48]:
# Save the project
aprx.save()


### <font color="blue">**5.2. Map Layer Time Settings Configuration**</font>

In this subsection we will configure the time settings for the map layers, so that the map view will be time-enabled

> <font color="orange">⚠️ **Reminder**:</font> When doing an update on the raw data, make sure you update the the end-time date for the project (this changes with each update).
- Current End Date: 9/30/2024, thus the time settings for the map layers will be set to the end of this date (9/30/2024, 23:59:59, or better, 10/1/2024, 00:00:00).

Set time settings configuration for the map layers

In [49]:
# Define the key time parameters for the layers using a dictionary
time_settings = {
    "st": datetime(2013, 1, 1, 0, 0),
    "et": datetime(2024, 10, 1, 0, 0),
    "td": datetime(2024, 10, 1, 0, 0) - datetime(2013, 1, 1, 0, 0),
    "stf": "date_datetime",
    "tsi": 1.0,
    "tsiu": "months",
    "tz": arcpy.mp.ListTimeZones("*Pacific*")[0]
}  
# where, st: start time, et: end time, td: time extent, stf: time field, 
# tsi: time interval, tsiu: time units, tz: time zone

Enable layer time, if needed

In [50]:
# Iterate through the data layer list and enable time
for mylyr in data_lyrs_list:
    # if the layer is not time-enabled
    if not mylyr.isTimeEnabled:
        # Enable time for the layer
        mylyr.enableTime("date_datetime", "", "TRUE", None)
        # Set the start time for the layer
        mylyr.time.startTime = time_settings["st"]
        # Set the end time for the layer
        mylyr.time.endTime = time_settings["et"]
        # Set the time field for the layer
        mylyr.time.startTimeField = time_settings["stf"]
        # Set the time step interval for the layer
        mylyr.time.timeStepInterval = time_settings["tsi"]
        # Set the time step interval units for the layer
        mylyr.time.timeStepIntervalUnits = time_settings["tsiu"]
        # Set the time zone for the layer
        mylyr.time.timeZone = time_settings["tz"]


Reset time step interval parameters (seems to be a problem showing 10 months instead of 1 month)

In [51]:
# Iterate through the data layer list and manually change the time's time step interval
for mylyr in data_lyrs_list:
    if mylyr.isTimeEnabled:
        mylyr.time.timeStepInterval = 1.0
        mylyr.time.timeStepIntervalUnits = "months"


Read and display the current time settings of the layers

In [52]:
# Iterate through the data layer list and display the time settings
for mylyr in data_lyrs_list:
    print(mylyr.name)
    print(f"- Time Field: {codebook[time_settings['stf']]['label']} ({time_settings['stf']})")
    print(f"- Time Step Intervals: {int(mylyr.time.timeStepInterval)} {mylyr.time.timeStepIntervalUnits}")
    print(f"- Start Time: {str(time_settings['st'].strftime('%m/%d/%Y %H:%M %p'))}")
    print(f"- End Time: {str(time_settings['et'].strftime('%m/%d/%Y %H:%M %p'))}")
    print(f"- Time Extent: {int(time_settings['td'].days):,} days ({int((time_settings['td'].days/365)*12)} months)")
    print(f"- Time Zone: {time_settings['tz']}")


OCSWITRS Collisions
- Time Field: Crash Date and Time (date_datetime)
- Time Step Intervals: 1 months
- Start Time: 01/01/2013 00:00 AM
- End Time: 10/01/2024 00:00 AM
- Time Extent: 4,291 days (141 months)
- Time Zone: (UTC-08:00) Pacific Time (US & Canada)
OCSWITRS Crashes
- Time Field: Crash Date and Time (date_datetime)
- Time Step Intervals: 1 months
- Start Time: 01/01/2013 00:00 AM
- End Time: 10/01/2024 00:00 AM
- Time Extent: 4,291 days (141 months)
- Time Zone: (UTC-08:00) Pacific Time (US & Canada)
OCSWITRS Parties
- Time Field: Crash Date and Time (date_datetime)
- Time Step Intervals: 1 months
- Start Time: 01/01/2013 00:00 AM
- End Time: 10/01/2024 00:00 AM
- Time Extent: 4,291 days (141 months)
- Time Zone: (UTC-08:00) Pacific Time (US & Canada)
OCSWITRS Victims
- Time Field: Crash Date and Time (date_datetime)
- Time Step Intervals: 1 months
- Start Time: 01/01/2013 00:00 AM
- End Time: 10/01/2024 00:00 AM
- Time Extent: 4,291 days (141 months)
- Time Zone: (UTC-08:00) 

Save the ArcGIS pro project

In [53]:
# Save the project
aprx.save()


### <font color="blue">**5.3. Update Layer Views and File Lists**</font>

List project and directories, layers in contents view, and layer files in directories

In [55]:
# Display the project details
print(f"Project:\n- Name: {project_aprx}\n- Path: {project_aprx_path}\n- Layers Directory: {layers_path}\n")

# Display the layers in content view
print(f"Layers in Content View:\n- Collisions: {collisions_lyr}\n- Crashes: {crashes_lyr}\n- Parties: {parties_lyr}\n- Victims: {victims_lyr}\n- Roads: {roads_lyr}\n- US Census 2020 Blocks: {usc2020blk_lyr}\n- Cities: {cities_lyr}\n- Boundaries: {boundaries_lyr}\n")

# Layers in Directory
print(f"Layers in Directory:\n- Collisions: {collisions_path}\n- Crashes: {crashes_path}\n- Parties: {parties_path}\n- Victims: {victims_path}\n- Roads: {roads_path}\n- US Census 2020 Blocks: {usc2020blk_path}\n- Cities: {cities_path}\n- Boundaries: {boundaries_path}")


Project:
- Name: AGPSWITRS.aprx
- Path: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.aprx
- Layers Directory: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\Layers

Layers in Content View:
- Collisions: OCSWITRS Collisions
- Crashes: OCSWITRS Crashes
- Parties: OCSWITRS Parties
- Victims: OCSWITRS Victims
- Roads: OCSWITRS Roads
- US Census 2020 Blocks: OCSWITRS Census 2020 Blocks
- Cities: OCSWITRS Cities
- Boundaries: OCSWITRS Boundaries

Layers in Directory:
- Collisions: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawData\collisions
- Crashes: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawData\crashes
- Parties: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawData\parties
- Victims: C:\Users\ktalexan\OneDrive - County of Orange\Documents\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\RawData\victims
- Road

## <font color="orangered">**6. Symbology Operations**</font>

### <font color="blue">**6.1. Data Layer Symbologies**</font>

Define symbology for each of the data layers (Crashes, Parties, Victims, and Collisions). The symbology is pre-saved in the SWITRS layer file, stored in the project's layer folder.

In [59]:
# Apply the symbology for the Collisions data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = collisions_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Collisions.lyrx"),
    symbology_fields = "VALUE_FIELD coll_severity coll_severity",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Collisions'>

In [60]:
# Apply the symbology for the Crashes data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = crashes_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Crashes.lyrx"),
    symbology_fields = "VALUE_FIELD coll_severity coll_severity",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Crashes'>

In [61]:
# Apply the symbology for the Parties data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = parties_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Parties.lyrx"),
    symbology_fields="VALUE_FIELD coll_severity coll_severity",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Parties'>

In [62]:
# Apply the symbology for the Victims data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = victims_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Victims.lyrx"),
    symbology_fields="VALUE_FIELD coll_severity coll_severity",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Victims'>

Define symbology for the supporting data layers (Cities, Roads, Boundaries). The symbology for each of the supporting data layers is pre-saved in the CITIES and ROADS layer files respectively, stored in the project's layer folder.

In [63]:
# Apply the symbology for the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = roads_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Roads.lyrx"),
    symbology_fields = "VALUE_FIELD road_cat road_cat",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Roads'>

In [64]:
# Apply the symbology for the US Census 2020 Blocks data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = usc2020blk_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Census 2020 Blocks.lyrx"),
    symbology_fields="VALUE_FIELD population_total population_total;EXCLUSION_CLAUSE_FIELD population_total population_total",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Census 2020 Blocks'>

In [66]:
# Apply the symbology for the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = cities_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Cities.lyrx"),
    symbology_fields="VALUE_FIELD city_pop_dens city_pop_dens;COLOR_EXPRESSION_FIELD city_pop_dens city_pop_dens",
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Cities'>

In [67]:
# Apply the symbology for the Boundaries data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = boundaries_lyr,
    in_symbology_layer = os.path.join(layers_path, "OCSWITRS Boundaries.lyrx"),
    symbology_fields = None,
    update_symbology = "MAINTAIN"
)

<Result 'OCSWITRS Boundaries'>

Save the ArcGIS pro project

In [69]:
# Save the project
aprx.save()

### <font color="blue">**6.2. Pop-Ups Configuration (Arcade)**</font>

Run these code segments below after opening the ArcGIS Pro project to configure the pop-ups for the feature classes.

**<font color="orange">Arcade Expression:</font> Configure Pop-Ups for the Crashes layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupCrashes`
- Expression:
    ```javascript
    var c = $feature.cid
    var ct = $feature.city
    var dt = Text($feature.date_datetime, 'M/D/YYYY H:m A')
    "Crash ID: " + c + " | Date: " + dt + ", " + ct
    ```
    

**<font color="orange">Arcade Expression:</font> Configure Pop-Ups for the Parties layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupParties`
- Expression:
    ```javascript
    var c = $feature.cid
    var p = $feature.party_number
    var dt = Text($feature.date_datetime, 'M/D/YYYY H:m A')
    "Crash ID: " + c + ", Party: " + p + " | Date: " + dt
    ```


**<font color="orange">Arcade Expression</font> Configure Pop-Ups for the Victims layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupVictims`
- Expression:
    ```javascript
    var c = $feature.cid
    var p = $feature.party_number
    var v = $feature.victim_number
    var dt = Text($feature.date_datetime, 'M/D/YYYY H:m A')
    "Crash ID: " + c + ", Party: " + p + ", Victim: " + v + " | Date: " + dt
    ```

**<font color="orange">Arcade Expression</font> Configure Pop-Ups for the Collisions layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupCollisions`
- Expression:
    ```javascript
    var c = $feature.cid
    var p = $feature.party_number
    var v = $feature.victim_number
    var ct = $feature.city
    var dt = Text($feature.date_datetime, 'M/D/YYYY H:m A')
    var exp1 = "Crash ID: " + c + ", Party: " + p + ", Victim: " + v + " | Date: " + dt + ", " + ct
    var exp2 = "Crash ID: " + c + ", Party: " + p + ", No Victim | Date: " + dt + ", " + ct
    if(!isEmpty(p) && !isEmpty(v)){
        return exp1
    } else if(!isEmpty(p) && isEmpty(v)){
        return exp2
    }
    ```
    

----

> <font color="coral">💬 **Note**:</font> <font color="darkgreen"> This is the end of data pre-processing (Notebook Part 2). The ArcGIS Online (AGO) processing of the data is continuing on a new ArcGIS notebook (Part 3)</font>